In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import csv
import pickle

In [ ]:
with open('test_data.csv', encoding="utf8", errors='ignore') as f:
      test_data = f.read()


In [ ]:
##please run the below hashed codes if you are using the livetwitter data
##we need to format the data to a DataFrame incase we are using the livetwitter feed
#test_data= test_data.replace('{"tweet":', " ")

In [ ]:
# def Convert(str1): 
#     li = list(str1.split("\n")) 
#     return li 
# str1 = test_data
# test_data=Convert(str1)

In [ ]:
# test_data=pd.DataFrame(test_data)
# test_data.columns=["tweet"]


In [ ]:
def remove_pattern(input_txt,pattern):
    r=re.findall(pattern,input_txt)
    for i in r:
        input_txt= re.sub(i,'',input_txt)
    return input_txt

In [ ]:
test_data['tweet']=np.vectorize(remove_pattern)(test_data['tweet'],"@[\w]*")
test_data['tweet'] = test_data['tweet'].str.replace("[^a-zA-Z#]", " ")
test_data['tweet']=test_data['tweet'].apply (lambda x: ' '.join(r for r in x.split() if len(r)>3))
tokenized_tweet= test_data['tweet'].apply(lambda x: x.split())



In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
tokenized_tweet.head()

In [ ]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

test_data['tweet'] = tokenized_tweet

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(test_data['tweet'])

In [ ]:
bow

In [ ]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))

prediction = loaded_model.predict_proba(bow)
prediction = prediction[:,1] >= 0.3 
prediction = prediction.astype(np.int)
test_data['predicted_label']=prediction


In [ ]:
test_data.to_csv('rf1024.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

all_words = ' '.join([text for text in test_data['tweet']])
from wordcloud import WordCloud

wordcloud = WordCloud(width=600, height=500, random_state=10, max_font_size=100).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:
positive_words = ' '.join([text for text in test_data['tweet'][test_data['predicted_label']==0]])

wordcloud = WordCloud(width=600, height=500, random_state=10, max_font_size=100).generate(positive_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:
negative_words = ' '.join([text for text in test_data['tweet'][test_data['predicted_label']==1]])

wordcloud = WordCloud(width=600, height=500, random_state=10, max_font_size=100).generate(negative_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:
def hashtag(x):
    hashtags=[]
    for i in x:
        r=re.findall(r'#(\w+)',i)
        hashtags.append(r)
    return hashtags

In [ ]:
hashtag_neutral=hashtag(test_data['tweet'][test_data['predicted_label']==0])
hashtag_negative=hashtag(test_data['tweet'][test_data['predicted_label']==1])

In [ ]:
hashtag_extract_neutral=sum(hashtag_neutral,[])
hashtag_extract_negative=sum(hashtag_negative,[])


In [ ]:
fd_neutral = nltk.FreqDist(hashtag_extract_neutral)
fd_table_neutral = pd.DataFrame({'Hashtag': list(fd_neutral.keys()),
                  'Count': list(fd_neutral.values())})

In [ ]:
fd_neutral_largest = fd_table_neutral.nlargest(columns="Count", n=10)
plt.figure(figsize=(13, 7))
ax = sns.barplot(data=fd_neutral_largest, x="Hashtag", y="Count")

In [ ]:
fd_negative = nltk.FreqDist(hashtag_extract_negative)
fd_table_negative = pd.DataFrame({'Hashtag': list(fd_negative.keys()),
                  'Count': list(fd_negative.values())})
fd_negative_largest = fd_table_negative.nlargest(columns="Count", n=10)
plt.figure(figsize=(13, 7))
ax = sns.barplot(data=fd_negative_largest, x="Hashtag", y="Count")